In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
np.random.seed(54321)

!pip install ipdb
import ipdb

!pip install ipython-autotime
%load_ext autotime

!pip install optuna
import optuna

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


  Created wheel for ipdb: filename=ipdb-0.13.2-cp36-none-any.whl size=10522 sha256=9c66a281c974c1cfb84ee6d85907570ed839e450e4aeaad45c3f6bff5eb19b13
  Stored in directory: /root/.cache/pip/wheels/60/c2/15/793365e3c9318c46ba914263740d90f1fe67f544b979141ce4
Successfully built ipdb
  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=97c2afce8e4ccb567e04eba40e86a4d4a508c738f7195126e005cc534b113808
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
     |████████████████████████████████| 184kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 10.8MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 8.2MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 8.0MB/s  eta 0:00:01
     |█████████████

In [2]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

# X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
# X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

time: 21.7 ms


In [7]:
# X_train=pd.read_csv('./data/train_data_preprocessing1.csv', sep=',') #we use this dataset to train our model
# Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
# X_test=pd.read_csv('./data/test_data_preprocessing1.csv', sep=',')

time: 756 µs


In [8]:
# X_test.head()

time: 517 µs


In [9]:
X_train.tail()

,Id,seq
1995,1995,TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...
1996,1996,ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...
1997,1997,TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...
1998,1998,CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...
1999,1999,TGCTAGTTGATGAAACAATAACTGCTAAAAGGTATACAGCCATGTC...


time: 10.1 ms


In [10]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)
time: 1.32 ms


In [11]:
# X_train['len'] = X_train.seq.apply(lambda x : len(x))

time: 796 µs


In [12]:
X_train.head()

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


time: 9.69 ms


In [13]:
import string
Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))

time: 1.17 ms


In [14]:
#  X_train['seq_1'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[0])
for i in range(0, 101, 1):
    X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
    X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])
    
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]]

#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+3])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+3])



time: 337 ms


In [15]:
X_train.head()

,Id,seq,seq_0,seq_1,seq_2,seq_3,seq_4,seq_5,seq_6,seq_7,seq_8,seq_9,seq_10,seq_11,seq_12,seq_13,seq_14,seq_15,seq_16,seq_17,seq_18,seq_19,seq_20,seq_21,seq_22,seq_23,seq_24,seq_25,seq_26,seq_27,seq_28,seq_29,seq_30,seq_31,seq_32,seq_33,seq_34,seq_35,seq_36,seq_37,...,seq_61,seq_62,seq_63,seq_64,seq_65,seq_66,seq_67,seq_68,seq_69,seq_70,seq_71,seq_72,seq_73,seq_74,seq_75,seq_76,seq_77,seq_78,seq_79,seq_80,seq_81,seq_82,seq_83,seq_84,seq_85,seq_86,seq_87,seq_88,seq_89,seq_90,seq_91,seq_92,seq_93,seq_94,seq_95,seq_96,seq_97,seq_98,seq_99,seq_100
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...,7,1,7,7,7,7,3,20,7,7,7,7,1,7,7,7,7,7,3,20,7,7,3,3,3,1,7,1,7,7,3,1,3,3,1,7,1,3,...,20,7,7,7,7,3,20,7,3,3,3,20,7,3,3,1,3,3,20,7,3,20,7,7,3,3,7,3,20,3,3,20,7,7,20,7,7,3,1,7
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...,3,7,7,3,3,20,7,7,7,7,7,3,3,1,3,1,20,7,20,7,1,7,20,7,3,20,20,1,3,3,20,7,20,7,20,7,7,7,...,20,3,1,7,7,3,1,7,3,1,7,3,20,7,7,7,20,1,3,20,1,1,1,7,7,7,7,3,3,20,7,20,3,3,1,7,7,20,7,7
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...,7,1,3,1,1,3,7,3,3,7,3,20,7,20,3,1,7,3,3,7,3,3,20,20,3,7,1,3,20,3,1,3,3,20,7,7,7,1,...,20,1,3,3,20,7,7,7,1,1,20,3,20,7,7,7,3,1,20,3,3,3,7,7,7,3,20,3,3,3,3,7,1,1,7,20,7,3,7,20
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...,7,3,3,20,3,3,3,20,20,7,7,3,1,3,3,1,3,7,7,7,1,7,1,3,3,1,7,20,20,20,20,7,7,1,7,7,7,7,...,3,1,3,20,7,20,3,1,7,7,1,1,7,7,3,20,7,1,1,7,20,20,20,3,7,7,7,7,3,1,7,1,7,20,7,3,20,1,1,1
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...,7,3,1,3,20,1,3,20,1,3,1,3,3,3,1,20,20,7,3,20,7,20,1,1,20,1,7,20,1,1,7,20,7,3,3,7,7,20,...,7,20,3,3,20,7,7,7,1,1,1,20,1,3,1,3,1,3,20,3,1,3,1,3,20,20,7,20,7,20,1,3,20,7,20,20,1,3,3,3


time: 141 ms


## What about counting number of different caracters 

In [16]:
# A, C, G, T

time: 582 µs


In [17]:
# X_train.seq.apply(lambda x : np.unique(x[:2]))


time: 552 µs


In [18]:
X = X_train.drop(['seq', 'Id'], axis=1)
X_t = X_test.drop(['seq', 'Id'], axis=1)
y = Y_train.Bound

time: 7.17 ms


## Models

In [19]:
# class LogisticRegressionBinary():
#     def __init__(self, lr=0.1, num_iter=100000, batch_size=1, verbose=False):
#         self.lr = lr
#         self.num_iter = num_iter
#         self.batch_size = batch_size
#         self.verbose = verbose
    
#     def __add_intercept(self, X):
#         intercept = np.ones((X.shape[0], 1))
#         return np.concatenate((intercept, X), axis=1)
    
#     def __sigmoid_func(self, z):
#         return 1 / (1 + np.exp(-z))
    
#     def __loss(self, h, y):
#         return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
#     def fit(self, X, y):
#         y = self.trans_y(y)
#         X = self.__add_intercept(X)
#         self.theta = np.zeros(X.shape[1])
        
#         for i in range(self.num_iter):
#             z = np.dot(X, self.theta)
#             h = self.__sigmoid_func(z)
                        
#             rand = np.random.choice(y.size, self.batch_size).squeeze()
#             gradient = np.dot(X[rand].T, (h[rand] - y[rand]))/y.size   
        
#             self.theta -= self.lr * gradient
#             #print('theta and grad',self.theta.shape ,  gradient.shape )
#             if(self.verbose == True and i % 100 == 0):
#                 z = np.dot(X, self.theta)
#                 h = self.__sigmoid(z)
#                 print(f'loss: {self.__loss(h, y)} \t')
    
#     def predict_probability(self, X):
#         X = self.__add_intercept(X)
    
#         return self.__sigmoid_func(np.dot(X, self.theta))
    
#     def predict(self, X, threshold=.5):
#           return np.where(self.predict_probability(X) >= 0.5, 1, 0)
        
          
#     def Accuracy_check(self,X,y):
#         return np.mean(self.predict(X)==y)
    
#     def trans_y(self, y):
#         if isinstance(y, pd.Series):
#             y = y.values
#         if isinstance(y, list):
#             y = np.array(y)
#         return y

time: 4.65 ms


In [20]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 125 ms


# Kernel

In [21]:
import numpy as np

### Functions for you to fill in ###

def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    kernel_matrix = (X.dot(Y.T) + c)**p
    
    return kernel_matrix


def rbf_kernel_2(X, Y, gamma):
    """
        Compute the Gaussian RBF kernel between two matrices X and Y::
            K(x, y) = exp(-gamma ||x-y||^2)
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            gamma - the gamma parameter of gaussian function (scalar)

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    n, d = X.shape
    m = Y.shape[0]
    
    kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
    kernel_matrix = np.exp(-gamma*kernel_matrix)
    
    return kernel_matrix


def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

# def laplace(X1, X2, alpha=10):
#     return np.exp(-alpha*np.abs(X1-X2))

# def polynomial(X1, X2, d=2):
#     return (X1.dot(X2.T) +1)**d

def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return X1.dot(X2.T)

def quadratic_kernel(X1, X2, power=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the quadratic kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    return (1 + linear_kernel(X1, X2))**power

def rbf_poly_kernel(X1, X2, sigma=10, d=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    
    return K+(X1.dot(X2.T) +1)**d

time: 60.5 ms


In [22]:
class ksolveRR_2():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
        self.sample_weights = sample_weights
            
    
    def fit(self):
        if self.sample_weights is not None:
            self.X *= self.sample_weights[:, None]
        
        X = self.X
        y = self.y
        lam = self.lam
        
        n, p = X.shape
        assert (len(y) == n)
        
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 20.6 ms


In [23]:
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = kernel
            
    
    def fit(self):
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
            
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 20.1 ms


In [24]:
import ipdb
# Logistic Ridge Regression (LRR)
class ksolveLRR():
    def __init__(self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.lam = lam
        
        self.kernel = kernel
        
        self.sigma = sigma
        self.max_iter = max_iter
        self.tol = tol
        
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)

        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        K = self.kernel(X, X, self.sigma)

        # Initialize
        alpha = np.zeros(n)
        
        # Hint: Use IRLS
        for n_iter in range(self.max_iter):
            alpha_old = alpha
            f = K.dot(alpha_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*(f))
            
            alpha = ksolveRR_2(X, y, lam = 2*self.lam, \
                               sigma=self.sigma, sample_weights = w).fit().alpha
            
            # Break condition (achieved convergence)
            if np.sum((alpha-alpha_old)**2) < self.tol:
                break  
                
                
        self.n_iter = n_iter
        self.alpha = alpha
        
        return self
    
        
    def predict(self, X, threshold):
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 46 ms


In [25]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'

import cvxopt

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    #cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(*cvx_matrices, options={'show_progress': False})
    return np.array(solution['x']).flatten()

solve_qp = cvxopt_qp

def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)
    return np.array(solution['x']).flatten()

solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]

def svm_dual_soft_to_qp_kernel(K, y, C=1):
    n = K.shape[0]
    assert (len(y) == n)
        
    # Dual formulation, soft margin
    P = np.diag(y).dot(K).dot(np.diag(y))
    # As a regularization, we add epsilon * identity to P
    eps = 1e-12
    P += eps * np.eye(n)
    q = - np.ones(n)
    G = np.vstack([-np.eye(n), np.eye(n)])
    h = np.hstack([np.zeros(n), C * np.ones(n)])
    A = y[np.newaxis, :]
    A = A.astype('float')
    b = np.array([0.])
    return P, q, G, h, A, b

# SVM primal soft
class KernelSVM():
    def __init__(self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel):
        self.alpha = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.C = C
        self.kernel = kernel
        self.lam = lam        
        self.sigma = sigma
        self.tol = tol
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)
        K = self.kernel(X, X, self.sigma)
        
        # Solve dual problem
        self.alpha = solve_qp(*svm_dual_soft_to_qp_kernel(K, y, C=self.C))
        
        
       # Compute support vectors and bias b
        sv = np.logical_and((self.alpha>self.tol), (self.C - self.alpha > self.tol))
        self.bias = y[sv] - K[sv].dot(self.alpha*y)
        self.bias =  self.bias.mean()

        self.support_vector_indices = np.nonzero(sv)[0]
        
        return self
        
        
        
    def predict(self, X, threshold):
#         y_pred = self.kernel(X, self.X_).dot(self.alphas* seld.y_)
        K_x = self.kernel(X, self.X, self.sigma)
        return np.where((K_x.dot(self.alpha * self.y) +  self.bias) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

time: 130 ms


In [26]:
# # Logistic Ridge Regression (LRR)
# class perceptron():
#     def __init__(self, X, y, iteration=100):
#         self.theta = np.zeros(len(X))
#         self.X = X
#         self.y = y
#         self.iteration = iteration
#         self.theta_0 = 0
    
#     def fit(self):
        
#         X = self.X
#         y = self.y
        
#         n, p = X.shape
#         assert (len(y) == n)
        
#         for it in range(self.iteration):
#             for i in range(len(y)):
#                 if y[i]*(np.dot(self.theta, X[i]) + self.theta_0)<= 0 :
#                     self.theta = self.theta + (np.dot(y[i], X[i]))
#                     self.theta_0 += y[i]
    
        
#     def predict(self, X, threshold):
#         return np.where(X.dot(self.self.theta)+self.theta_0 >= threshold, 1, 0)
        
          
#     def Accuracy_check(self,X, y, threshold=.5):
#         return np.mean(self.predict(X, threshold)==y)

time: 2.6 ms


# Cross Validation

In [27]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TransformerM
# from sklearn.feature_extraction.text import TfidfTransformer

time: 154 ms


In [28]:
# vectorizer = CountVectorizer(max_features=2000)
# vectorizer.fit_transform(X[:10])
# # vectorizer.get_feature_names()

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

time: 1.32 ms


In [29]:
kfold=KFold(n_splits=5)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# X_cross = X.values
# X_t = X_t.values

y_cross = y.values

# vectorizer = TfidfVectorizer()
# X_cross = vectorizer.fit_transform(X)

# X_cross = onehot_encoder.fit_transform(X.values[:, :101])
# X_t_enc = onehot_encoder.fit_transform(X_t.values[:, :101])

X_cross = onehot_encoder.fit_transform(X)
X_t_enc = onehot_encoder.fit_transform(X_t)


# X_cross = transformer.fit_transform(X.values)
# X_t_enc = transformer.fit_transform(X_t.values)


# scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
# scaler.fit(X_cross)

# X_cross = scaler.transform(X_cross)

time: 113 ms


In [30]:
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.feature_selection import SelectKBest, chi2

time: 1.02 ms


In [31]:
# def feature_selector(X):
#     #sel = VarianceThreshold()
#     sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
#     sel.fit_transform(X)
#     return X
# def best_feature_selector(X,y,num_features=50):
#     #print(X.shape)
#     features = SelectKBest(chi2, k=num_features).fit(X, y)
#     #print(X_new.shape)
#     return features

time: 2.3 ms


In [32]:
X.shape

(2000, 101)

time: 3.58 ms


In [33]:
# trans = best_feature_selector(X, y, 30)

time: 640 µs


In [34]:
# X = trans.transform(X)
# X_t = trans.transform(X_t)

time: 589 µs


In [35]:
X.values.shape

(2000, 101)

time: 2.45 ms


In [36]:
X_t.values.shape

(1000, 101)

time: 2.99 ms


In [37]:
X_t_enc.shape

(1000, 404)

time: 2.6 ms


In [38]:
X_cross.shape

(2000, 404)

time: 1.99 ms


In [39]:
# PRUNNING TRIAL 

# def objective_sgd(trial):
# #     c  = trial.suggest_loguniform('c', 1e-20, 30)
#     sigma  = trial.suggest_loguniform('sigma', 1e-3, 20) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
# #     kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
#     lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
# #     tol = trial.suggest_loguniform('tol', 1e-10, 1e-0)
    
#     # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
#     # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
#     # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
#     # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
# #     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
# #               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'} 
# #     
#     kenel = rbf_kernel
#     models = {ksolveRR: 'k Ridge Reg'} 
      
# #     accuracy = []
#     for model in models:
#         accuracy = []
#         for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
#             X_train, y_train = X_cross[train_index], y_cross[train_index]
#             X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
            
            
# #             ipdb.set_trace()
#             if models[model] == 'weigh Ridge Reg':
#                 sample_weights = np.random.rand(len(y_train))
#                 model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
                
#             elif models[model] == 'k Logistic Ridge Reg':
#                 model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=100, tol = tol, kernel = kenel)
#             elif models[model] == 'k Ridge Reg':
#                 model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
#             else:
#                 model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
#             model_curr.fit()
            
#             acc = model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5)
#             accuracy.append(acc)
            
#             intermediate_value = acc
#             trial.report(intermediate_value, i)
        
# #             print(f'accurracy fold {i}: {accuracy[i]}')
        
# #         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

#     return np.mean(accuracy)

# # Set up the median stopping rule as the pruning condition.
# # study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
# study = optuna.create_study(pruner=optuna.pruners.MedianPruner(), sampler=sampler, direction='maximize')

# # sampler = optuna.samplers.TPESampler()    # sampler = SimulatedAnnealingSampler()
# # study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective_sgd, n_trials=1500,show_progress_bar=True)


# trial = study.best_trial

# print('Accuracy: {}'.format(trial.value))
# print("Best hyperparameters: {}".format(trial.params))

time: 7.06 ms


In [49]:
2e1

20.0

time: 1.96 ms


In [ ]:
def objective_sgd(trial):
    c  = trial.suggest_loguniform('c', 1e-20, 2e1)
    sigma  = trial.suggest_float('sigma', 1e-20, 3e1) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
    kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
    lam = trial.suggest_float('lam', 1e-20, 1e-0)
#     tol = trial.suggest_loguniform('tol', 1e-6, 1e-0)
    
    model = trial.suggest_categorical('models', [ksolveRR , ksolveRR_2, ksolveLRR, KernelSVM])
    
    # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
    # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
    # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
    # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
    tol = 1e-6
    models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
              ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'} 
      
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]


#             ipdb.set_trace()
        if models[model] == 'weigh Ridge Reg':
            sample_weights = np.random.rand(len(y_train))
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)

        elif models[model] == 'k Logistic Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=500, tol = tol, kernel = kenel)
        elif models[model] == 'k Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
        else:
            model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)

        model_curr.fit()
        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
#             print(f'accurracy fold {i}: {accuracy[i]}')

#         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

    return np.mean(accuracy)

sampler = optuna.samplers.TPESampler()

study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective_sgd, n_trials=5500,show_progress_bar=True)


trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7f01e02d1d90> which is of type function.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7f01e02d1950> which is of type function.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR'> which is of type type.

/usr/local/lib/python3.6/dist-packages/optuna/distributions.py:331: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for pe

[W 2020-05-29 08:56:55,147] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7f01e02d1d90> which is of type function.
[W 2020-05-29 08:56:55,149] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7f01e02d1950> which is of type function.
[W 2020-05-29 08:56:55,153] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR'> which is of type type.
[W 2020-05-29 08:56:55,154] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR_2'> which is of type type.
[W 2020-05-29 08:56:55,155] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent st

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: RuntimeWarning:

invalid value encountered in greater_equal



     pcost       dcost       gap    pres   dres
 0: -4.0650e+02 -2.2737e-04  7e+03  8e+01  1e-16
 1: -4.0651e+00 -2.2737e-04  7e+01  8e-01  4e-16
 2: -4.0703e-02 -2.2737e-04  7e-01  8e-03  2e-16
 3: -4.5920e-04 -2.2723e-04  7e-03  8e-05  3e-16
 4: -5.7084e-05 -2.1374e-04  2e-04  8e-07  2e-16
 5: -6.0459e-05 -7.0612e-05  1e-05  2e-08  6e-16
 6: -6.5813e-05 -6.5923e-05  1e-07  2e-10  3e-16
 7: -6.5866e-05 -6.5868e-05  1e-09  2e-12  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.9150e+02 -2.2616e-04  7e+03  8e+01  1e-16
 1: -3.9151e+00 -2.2616e-04  7e+01  8e-01  3e-16
 2: -3.9201e-02 -2.2616e-04  7e-01  8e-03  2e-16
 3: -4.4225e-04 -2.2601e-04  7e-03  8e-05  5e-16
 4: -5.4978e-05 -2.1231e-04  2e-04  8e-07  4e-16
 5: -5.8229e-05 -6.8320e-05  1e-05  2e-08  7e-16
 6: -6.3385e-05 -6.3494e-05  1e-07  2e-10  3e-16
 7: -6.3436e-05 -6.3437e-05  1e-09  2e-12  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.0200e+02 -2.2701e

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning:

overflow encountered in exp



[I 2020-05-29 08:57:58,242] Finished trial#6 with value: 0.501 with parameters: {'c': 6.785591849807885, 'sigma': 2.3306372896112593, 'kenel': <function quadratic_kernel at 0x7f01e02d1950>, 'lam': 0.04147298022505164, 'models': <class '__main__.ksolveLRR'>}. Best is trial#3 with value: 0.5780000000000001.
[W 2020-05-29 08:57:58,247] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7f01e02d1d90> which is of type function.
[W 2020-05-29 08:57:58,248] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7f01e02d1950> which is of type function.
[W 2020-05-29 08:57:58,250] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR'> which is of type type.
[W 2020-05-29 08:57:58,251] Choices f

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in true_divide



[I 2020-05-29 08:58:30,781] Finished trial#15 with value: 0.501 with parameters: {'c': 4.221508592389299e-06, 'sigma': 9.591550257739033, 'kenel': <function quadratic_kernel at 0x7f01e02d1950>, 'lam': 0.4605360299587907, 'models': <class '__main__.ksolveLRR'>}. Best is trial#13 with value: 0.5955.
[W 2020-05-29 08:58:30,789] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function rbf_kernel at 0x7f01e02d1d90> which is of type function.
[W 2020-05-29 08:58:30,790] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function quadratic_kernel at 0x7f01e02d1950> which is of type function.
[W 2020-05-29 08:58:30,794] Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class '__main__.ksolveRR'> which is of type type.
[W 2020-05-29 08:58:30,795] Choices for a cat

In [ ]:
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
def objective_sgd(trial):
#     c  = trial.suggest_loguniform('c', 1e-20, 2e1)
    sigma  = trial.suggest_float('sigma', 1e-25, 25) # trial.suggest_float('sigma', 1e-5, 1e-3, log=True)
#     kenel = trial.suggest_categorical('kenel', [rbf_kernel, quadratic_kernel])
    lam = trial.suggest_float('lam', 1e-30, 1e-0)
    tol = trial.suggest_loguniform('tol', 1e-10, 1e-0)
    
    # ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
    # ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
    # ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
    # KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel
    
#     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
#               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'} 
#     
    kenel = rbf_kernel
    models = {ksolveRR : 'k Ridge Reg} 
      
#     accuracy = []
    for model in models:
        accuracy = []
        for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):
            X_train, y_train = X_cross[train_index], y_cross[train_index]
            X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]
            
            
#             ipdb.set_trace()
            if models[model] == 'weigh Ridge Reg':
                sample_weights = np.random.rand(len(y_train))
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
                
            elif models[model] == 'k Logistic Ridge Reg':
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=500, tol = tol, kernel = kenel)
            elif models[model] == 'k Ridge Reg':
                model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
            else:
                model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
            model_curr.fit()
            accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
#             print(f'accurracy fold {i}: {accuracy[i]}')
        
#         print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

    return np.mean(accuracy)

sampler = optuna.samplers.TPESampler()

study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective_sgd, n_trials=5500,show_progress_bar=True)


trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
# Augmented data

# sigma  = trial.suggest_loguniform('sigma', 1e-20, 20)
# lam = trial.suggest_loguniform('lam', 1e-20, 1e-1)

# models = {ksolveRR: 'k Ridge Reg'} (Augmented data)
    # kenel = quadratic_kernel
    # Accuracy: 0.6555
    # Best hyperparameters: {'sigma': 4.065620491225982, 'lam': 0.013374107290659768}
    # time: 49min 21s
    #----
    # kenel = rbf_kernel
    # Accuracy: 0.6575 # 0.6575000000000001
    # Best hyperparameters: {'sigma': 4.119788517147901, 'lam': 1.2752298700618223e-14}, {'sigma': 4.063158315715049, 'lam': 8.30834772639223e-05}
    # time: 32min 57s
    
    # ------
    # kenel = rbf_kernel
    # Accuracy: 0.6595000000000001
    # Best hyperparameters: {'sigma': 4.1032895001889464, 'lam': 0.00014526367793975609}
    # time: --

    
# sigma  = trial.suggest_loguniform('sigma', 1e-3, 20)
# lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)

# models = {ksolveRR: 'k Ridge Reg'} (Non-Augmented data)
    # kenel = quadratic_kernel
    # Accuracy: 0.6555
    # Best hyperparameters: {'sigma': 4.069738272304652, 'lam': 0.053817561640154984}
    # time: 48min 12s
    #----
    # kenel = rbf_kernel
    # Accuracy: 0.6575000000000001
    # Best hyperparameters: {'sigma': 4.001185698777986, 'lam': 1.9770379950513775e-10}
    # time: 29min 58s
    


# Original data

# sigma  = trial.suggest_loguniform('sigma', 1e-3, 20)
# lam = trial.suggest_loguniform('lam', 1e-15, 1e-0)
  
# models = {ksolveRR: 'k Ridge Reg'} (Non-Augmented data standardize)
    # kenel = rbf_kernel
    # Accuracy: 0.612
    # Best hyperparameters: {'sigma': 2.8925324917718167, 'lam': 1.2575244169567934e-08}
    # time: 28min 33s 
    #----
    # kenel = quadratic_kernel
    # Accuracy: EROOR LinearAlg
    # Best hyperparameters: 
    # time: 

In [39]:
#     models = {ksolveRR : 'k Ridge Reg', ksolveRR_2: 'weigh Ridge Reg', \
#               ksolveLRR: 'k Logistic Ridge Reg', KernelSVM : 'Kernal SVM'

# c  = 
sigma  = 4.1032895001889464
kenel = rbf_kernel
lam = 0.00014526367793975609
# tol = 
    
#     ksolveRR (self, X, y, lam= 0.0001, sigma=0.5, kernel=rbf_kernel)
#     ksolveRR_2 (self, X, y, lam= 0.0001, sigma=0.5, sample_weights = None, kernel = rbf_kernel
#     ksolveLRR (self, X, y, lam = 0.1, sigma = 4, max_iter=100, tol=1e-5, kernel=rbf_kernel
#     KernelSVM (self, X, y, C=0.1, lam = 0.1, sigma = 4, tol = 1e-1, kernel=rbf_kernel

    
models = {ksolveRR : 'k Ridge Reg'}

for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if models[model] == 'weigh Ridge Reg':
            sample_weights = np.random.rand(len(y_train))
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, sample_weights = sample_weights, kernel = kenel)
        elif models[model] == 'k Logistic Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, max_iter=500, tol = tol, kernel = kenel)
        
        elif models[model] == 'k Ridge Reg':
            model_curr = model(X_train, y_train, lam = lam, sigma = sigma, kernel = kenel)
        else:
            model_curr = model(X_train, y_train, C=c, lam = lam, sigma = sigma, tol= tol, kernel = kenel)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.6125
accurracy fold 1: 0.655
accurracy fold 2: 0.6825
accurracy fold 3: 0.655
accurracy fold 4: 0.6925

Average accuracy k Ridge Reg is : 0.6595000000000001

time: 3.86 s


In [56]:
# 0.6535 = 0.68800
# 0.657 = 0.69200

time: 525 µs


In [58]:
# # Cehckinf full model
# model = ksolveRR(X_cross, y_cross, lam = lam, sigma = sigma, kernel = kenel)
# # model = svm_primal_soft_to_qp(X_cross, y_cross, C=1)

# model.fit()

# model.Accuracy_check(X_cross, y_cross, threshold=0.5)

time: 881 µs


# Predictions

In [40]:
model = ksolveRR(X_cross, y_cross, lam = lam, sigma = sigma, kernel = kenel)
model.fit()
y_pred = model.predict(X_t_enc, 0.5)

time: 1.24 s


In [41]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

,Id
0,0
1,1
2,2
3,3
4,4


time: 30.2 ms


In [42]:
sample['Bound'] = y_pred

time: 2.14 ms


In [43]:
sample.tail()

,Id,Bound
995,995,0
996,996,0
997,997,0
998,998,1
999,999,1


time: 15.6 ms


In [44]:
sample.to_csv('./ksolveRR_0.6595_cv_rbf_kernel_sigma_'+str(sigma)+'_lam_'+str(lam)+'.csv', index=False)

time: 69.6 ms
